In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import pandas_datareader.data as web
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.metrics import brier_score_loss, roc_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler

/Users/xiaozhezhang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


| market_category | feature_name | id |
|-----------------|--------------|----|
| Bank            | bac          |  1 |
| Bank            | citi         |  2 |
| Commodity       | corn         |  3 |
| Currency        | euro         |  4 |
| Commodity       | gold         |  5 |
| Inflation       | infl5y       |  6 |
| Commodity       | iyr          |  7 |
| Currency        | pound        |  8 |
| Commodity       | silver       |  9 |
| Commodity       | soybns       | 10 |
| Equity          | sp12m        | 11 |
| Equity          | sp6m         | 12 |
| Commodity       | wheat        | 13 |
| Currency        | yen          | 14 |


Return Model (Log Price)

In [2]:
df = pd.read_csv("mpd_sp500.csv")

# drop columns that has "maturity_target" , "lg_change_decr", and "lg_change_incr" in the column name; those are irrelevant for feature selection
df = df[df.columns.drop(list(df.filter(regex='maturity_target')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_decr')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_incr')))]
df = df[df.columns.drop(list(df.filter(regex='SP_adj_close')))]

# drop SP_lg_ret(%)	
df = df.drop(['SP_lg_ret(%)'], axis=1)
# df = df.drop(['SP_lg_pr'], axis=1)
df = df.drop(['VIX'], axis=1)



In [3]:
# Generate lagged variables from f1_mu to SP_lg_pr
lags = 6
for lag in range(1, lags+1):
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_ret_vol')+1]:
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('VIX')+1]: 
    for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_pr')+1]: 
    # for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('SP_lg_ret(%)')+1]:    
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_3611/4064986733.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_3611/4064986733.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_3611/4064986733.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [4]:
df_lagged = df.copy()
# drop NA rows
df_lagged = df_lagged.dropna()
df_lagged.shape


(571, 890)

In [6]:
df_lagged.to_csv('mpd_sp500_lagged_log_price.csv', index=False)

In [7]:
df_lagged.head()

,Date,f1_mu,f1_sd,f1_skew,f1_kurt,f1_p10,f1_p50,f1_p90,f1_prDec,f1_prInc,...,f14_mu_lag6,f14_sd_lag6,f14_skew_lag6,f14_kurt_lag6,f14_p10_lag6,f14_p50_lag6,f14_p90_lag6,f14_prDec_lag6,f14_prInc_lag6,SP_lg_pr_lag6
6,2013-02-28,-0.00858,0.15972,-0.36374,0.77551,-0.21056,0.00001,0.18163,0.10981,0.08030,...,0.00061,0.05305,-0.23766,0.17369,-0.06872,0.00280,0.06621,0.03761,0.02347,7.300432
7,2013-03-07,-0.00402,0.15324,-0.46085,1.08979,-0.19590,0.00554,0.17463,0.09637,0.07220,...,0.00097,0.05578,-0.20982,0.28982,-0.07148,0.00258,0.06933,0.04333,0.03237,7.309761
8,2013-03-14,-0.00402,0.15324,-0.46085,1.08979,-0.19590,0.00554,0.17463,0.09637,0.07220,...,0.00097,0.05578,-0.20982,0.28982,-0.07148,0.00258,0.06933,0.04333,0.03237,7.311960
9,2013-03-21,-0.00628,0.15008,-0.50463,0.87713,-0.19770,0.00489,0.17013,0.09823,0.06551,...,0.00179,0.06249,-0.21652,0.41476,-0.07796,0.00383,0.07824,0.05847,0.05089,7.319461
10,2013-03-28,-0.00628,0.15008,-0.50463,0.87713,-0.19770,0.00489,0.17013,0.09823,0.06551,...,0.00179,0.06249,-0.21652,0.41476,-0.07796,0.00383,0.07824,0.05847,0.05089,7.327373


In [10]:
# Define the target variable
start_colunm = df_lagged.columns.get_loc('f1_mu_lag1')
# end_column = df_lagged.columns.get_loc('VIX_lag6')
end_column = df_lagged.columns.get_loc('SP_lg_pr_lag6')
# end_column = df_lagged.columns.get_loc('SP_lg_ret(%)_lag6')

column_index = list(range(start_colunm, end_column+1))

X = df_lagged.iloc[:, column_index]
# y = df_lagged['SP_lg_ret(%)'] 
y = df_lagged['SP_lg_pr'] 

split_index = int(len(X)*0.75)
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [12]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((428, 762), (143, 762), (428,), (143,))

In [18]:
# run a lasso regression to select features
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')
lassoCV.fit(X_train, y_train)

LassoCV(cv=10, max_iter=10000, random_state=12345, selection='random')

In [28]:
print("In Sample R^2: ", f'{lassoCV.score(X_train, y_train):.5f}')
print()
print("Out of Sample R^2: ", f'{lassoCV.score(X_test, y_test):.5f}')
print()
# lasso coefficients with corresponding feature names
lasso_coef = pd.DataFrame(lassoCV.coef_, index=X.columns, columns=['coef'])
lasso_coef = lasso_coef[lasso_coef.coef != 0]

print("Number of features selected: ", len(lasso_coef))
print(lasso_coef)

print()
# show the predicted value
lass_y_pred = lassoCV.predict(X_test)
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, lass_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, lass_y_pred)
lass_mape = np.mean(np.abs((y_test - lass_y_pred) / y_test)) * 100

print('Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')


In Sample R^2:  0.99395

Out of Sample R^2:  0.75795

Number of features selected:  32
                   coef
f2_kurt_lag1  -0.004843
f6_skew_lag1  -0.003889
f6_kurt_lag1   0.000321
f7_kurt_lag1   0.000022
f12_kurt_lag1  0.004235
f14_kurt_lag1 -0.009597
SP_lg_pr_lag1  0.851074
f5_skew_lag2   0.000034
f6_kurt_lag2   0.000196
f7_kurt_lag2   0.000787
f14_kurt_lag2  0.002386
SP_lg_pr_lag2  0.102479
f5_kurt_lag3   0.004254
f6_kurt_lag3   0.000454
f7_kurt_lag3  -0.000575
f14_skew_lag3  0.003035
f14_kurt_lag3  0.000120
f5_kurt_lag4   0.001617
f6_kurt_lag4   0.000683
f12_kurt_lag4 -0.000968
f14_kurt_lag4  0.010490
f1_kurt_lag5   0.003122
f6_kurt_lag5   0.000249
SP_lg_pr_lag5  0.006112
f3_kurt_lag6   0.005151
f6_kurt_lag6   0.000225
f7_kurt_lag6   0.000141
f8_kurt_lag6   0.004891
f9_skew_lag6  -0.000801
f11_kurt_lag6 -0.000282
f14_skew_lag6  0.001496
f14_kurt_lag6  0.001547

Test set evaluation:
MSE: 0.00120, RMSE: 0.03461, MAE: 0.02935, MAPE: 0.35101


Applied StandardScaler

In [21]:
# run a lasso regression to select features
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lassoCV2 = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')
lassoCV2.fit(X_train_scaled, y_train)

LassoCV(cv=10, max_iter=10000, random_state=12345, selection='random')

In [27]:
print("In Sample R^2: ", f'{lassoCV2.score(X_train_scaled, y_train):.5f}')
print()
print("Out of Sample R^2: ", f'{lassoCV2.score(X_test_scaled, y_test):.5f}')
print()

# lasso coefficients with corresponding feature names
lasso_coef = pd.DataFrame(lassoCV2.coef_, index=X.columns, columns=['coef'])
lasso_coef = lasso_coef[lasso_coef.coef != 0]

print("Number of features selected: ", len(lasso_coef))
print(lasso_coef)

print()
# show the predicted value
lass_y_pred = lassoCV2.predict(X_test_scaled)
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, lass_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, lass_y_pred)
lass_mape = np.mean(np.abs((y_test - lass_y_pred) / y_test)) * 100

print('Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')


In Sample R^2:  0.99338

Out of Sample R^2:  0.86058

Number of features selected:  16
                    coef
f2_skew_lag1    0.000281
f8_mu_lag1      0.000869
f14_kurt_lag1  -0.001718
SP_lg_pr_lag1   0.211871
SP_lg_pr_lag2   0.015468
f5_kurt_lag3    0.000183
f8_kurt_lag3    0.000103
f12_p50_lag3    0.000844
f14_skew_lag3   0.000813
f1_p90_lag4     0.001004
f6_kurt_lag4    0.000433
f11_prInc_lag4  0.000136
f14_kurt_lag4   0.002333
SP_lg_pr_lag5   0.002863
f2_prDec_lag6   0.001365
f8_kurt_lag6    0.000029

Test set evaluation:
MSE: 0.00069, RMSE: 0.02626, MAE: 0.02166, MAPE: 0.25922


##### Applied Standad Scaler Increase the of sample R^2 and decreased out os sample MSE, RMSE, MAE, and MAPE

In [ ]:
# # ridge regression
# from sklearn.linear_model import RidgeCV
# ridgeCV = RidgeCV(cv=10)
# ridgeCV.fit(X_train_scaled, y_train)

# print("In Sample R^2: ", f'{ridgeCV.score(X_train_scaled, y_train):.5f}')
# print()
# print("Out of Sample R^2: ", f'{ridgeCV.score(X_test_scaled, y_test):.5f}')
# print()
# ridge_coef = pd.DataFrame(ridgeCV.coef_, index=X.columns, columns=['coef'])
# ridge_coef = ridge_coef[ridge_coef.coef != 0]

# print("Number of features selected: ", len(ridge_coef))
# ridge_coef

# print()
# # test the model on testing data
# ridge_y_pred = ridgeCV.predict(X_test_scaled)

# # calculate the MSE, RMSE, and MAE
# ridge_mse = mean_squared_error(y_test, ridge_y_pred)
# ridge_rmse = np.sqrt(ridge_mse)
# ridge_mae = mean_absolute_error(y_test, ridge_y_pred)
# print(f'MSE: {ridge_mse:.5f}, RMSE: {ridge_rmse:.5f}, MAE: {ridge_mae:.5f}')


In [ ]:
# print("In Sample R^2: ", f'{lassoCV.score(X_train_scaled, y_train):.5f}')
# print()
# print("Out of Sample R^2: ", f'{lassoCV.score(X_test_scaled, y_test):.5f}')
# print()
# # lasso coefficients with corresponding feature names
# lasso_coef = pd.DataFrame(lassoCV.coef_, index=X.columns, columns=['coef'])
# lasso_coef = lasso_coef[lasso_coef.coef != 0]

# print("Number of features selected: ", len(lasso_coef))
# lasso_coef

In [ ]:
# # show the predicted value
# lass_y_pred = lassoCV.predict(X_test_scaled)
# # calculate the MSE, RMSE, and MAE
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# lass_mse = mean_squared_error(y_test, lass_y_pred)
# lass_rmse = np.sqrt(lass_mse)
# lass_mae = mean_absolute_error(y_test, lass_y_pred)
# print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}')

In [ ]:
# # ridge regression
# from sklearn.linear_model import RidgeCV
# ridgeCV = RidgeCV(cv=10)
# ridgeCV.fit(X_train_scaled, y_train)

# print("In Sample R^2: ", f'{ridgeCV.score(X_train_scaled, y_train):.5f}')
# print()
# print("Out of Sample R^2: ", f'{ridgeCV.score(X_test_scaled, y_test):.5f}')
# print()
# ridge_coef = pd.DataFrame(ridgeCV.coef_, index=X.columns, columns=['coef'])
# ridge_coef = ridge_coef[ridge_coef.coef != 0]

# print("Number of features selected: ", len(ridge_coef))
# ridge_coef

In [ ]:
# # test the model on testing data
# ridge_y_pred = ridgeCV.predict(X_test_scaled)

# # calculate the MSE, RMSE, and MAE
# ridge_mse = mean_squared_error(y_test, ridge_y_pred)
# ridge_rmse = np.sqrt(ridge_mse)
# ridge_mae = mean_absolute_error(y_test, ridge_y_pred)
# print(f'MSE: {ridge_mse:.5f}, RMSE: {ridge_rmse:.5f}, MAE: {ridge_mae:.5f}')

Volatility Model

In [29]:
df = pd.read_csv("mpd_sp500.csv")
# drop columns that has "maturity_target" , "lg_change_decr", and "lg_change_incr" in the column name; those are irrelevant for feature selection
df = df[df.columns.drop(list(df.filter(regex='maturity_target')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_decr')))]
df = df[df.columns.drop(list(df.filter(regex='lg_change_incr')))]
df = df[df.columns.drop(list(df.filter(regex='SP_adj_close')))]

# drop SP_lg_ret(%)	
df = df.drop(['SP_lg_pr'], axis=1)
# df2 = df2.drop(['SP_lg_ret(%)'], axis=1)

In [30]:
# Generate lagged variables from f1_mu to SP_lg_pr for df only
lags = 6
for lag in range(1, lags+1):
    for col in df.columns[df.columns.get_loc('f1_mu'):df.columns.get_loc('VIX')+1]:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_3611/1796792772.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_3611/1796792772.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_lag{lag}'] = df[col].shift(lag)
/var/folders/57/dq27lfpd1zb7kzzggckgkm2h0000gn/T/ipykernel_3611/1796792772.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

In [31]:
df2_lagged = df.copy()
# drop NA rows
df2_lagged = df2_lagged.dropna()
df_lagged.shape

(571, 890)

In [32]:
# Define the target variable
start_colunm = df2_lagged.columns.get_loc('f1_mu_lag1')
# end_column = df_lagged.columns.get_loc('VIX_lag6')
end_column = df2_lagged.columns.get_loc('VIX_lag6')

column_index = list(range(start_colunm, end_column+1))

X = df2_lagged.iloc[:, column_index]
y = df2_lagged['VIX'] 


split_index = int(len(X)*0.75)
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [34]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((412, 768), (138, 768), (412,), (138,))

In [37]:
lassoCV3 = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')
lassoCV3.fit(X_train, y_train)

LassoCV(cv=10, max_iter=10000, random_state=12345, selection='random')

In [41]:
print("In Sample R^2: ", f'{lassoCV3.score(X_train, y_train):.5f}')
print()
print("Out of Sample R^2: ", f'{lassoCV3.score(X_test, y_test):.5f}')
print()

# lasso coefficients with corresponding feature names
lasso_coef = pd.DataFrame(lassoCV3.coef_, index=X.columns, columns=['coef'])
lasso_coef = lasso_coef[lasso_coef.coef != 0]

print("Number of features selected: ", len(lasso_coef))
print(lasso_coef)

print()
# show the predicted value
lass_y_pred = lassoCV3.predict(X_test)
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, lass_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, lass_y_pred)
lass_mape = np.mean(np.abs((y_test - lass_y_pred) / y_test)) * 100

print('Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')

In Sample R^2:  0.87705

Out of Sample R^2:  -0.56624

Number of features selected:  102
                       coef
f1_kurt_lag1       0.010034
f2_kurt_lag1       0.017604
f3_skew_lag1      -0.000484
f4_skew_lag1       0.007961
f4_kurt_lag1       0.013543
...                     ...
f9_kurt_lag6      -0.003379
f10_kurt_lag6      0.007707
f12_skew_lag6      0.008682
f14_kurt_lag6      0.004494
SP_lg_ret(%)_lag6  0.002279

[102 rows x 1 columns]

Test set evaluation:
MSE: 0.00453, RMSE: 0.06731, MAE: 0.05493, MAPE: 30.99063


With Standard Scalar

In [40]:
# run a lasso regression to select features
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lassoCV4 = LassoCV(cv=10, random_state=12345, max_iter=10000, tol=0.0001, selection='random')
lassoCV4.fit(X_train_scaled, y_train)

LassoCV(cv=10, max_iter=10000, random_state=12345, selection='random')

In [42]:
print("In Sample R^2: ", f'{lassoCV4.score(X_train_scaled, y_train):.5f}')
print()
print("Out of Sample R^2: ", f'{lassoCV4.score(X_test_scaled, y_test):.5f}')
print()

# lasso coefficients with corresponding feature names
lasso_coef = pd.DataFrame(lassoCV4.coef_, index=X.columns, columns=['coef'])
lasso_coef = lasso_coef[lasso_coef.coef != 0]

print("Number of features selected: ", len(lasso_coef))
print(lasso_coef)

print()
# show the predicted value
lass_y_pred = lassoCV4.predict(X_test_scaled)
# calculate the MSE, RMSE, and MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error
lass_mse = mean_squared_error(y_test, lass_y_pred)
lass_rmse = np.sqrt(lass_mse)
lass_mae = mean_absolute_error(y_test, lass_y_pred)
lass_mape = np.mean(np.abs((y_test - lass_y_pred) / y_test)) * 100

print('Test set evaluation:')
print(f'MSE: {lass_mse:.5f}, RMSE: {lass_rmse:.5f}, MAE: {lass_mae:.5f}, MAPE: {lass_mape:.5f}')

In Sample R^2:  0.82069

Out of Sample R^2:  0.61970

Number of features selected:  26
                       coef
f1_kurt_lag1       0.001674
f2_kurt_lag1       0.000922
f4_p50_lag1       -0.000550
f6_skew_lag1       0.001394
f7_prInc_lag1      0.002923
f10_skew_lag1     -0.000508
f12_prDec_lag1     0.003667
f14_kurt_lag1      0.015564
VIX_lag1           0.041652
f1_kurt_lag2       0.001202
f3_kurt_lag2       0.000164
f7_kurt_lag2      -0.000033
VIX_lag2           0.002101
f2_p50_lag3       -0.001600
f8_p10_lag3        0.000111
f8_prInc_lag3     -0.002027
f2_p50_lag4       -0.000161
f9_p10_lag4       -0.000846
f6_skew_lag5       0.001256
f9_kurt_lag5       0.000029
f13_skew_lag5     -0.000218
SP_lg_ret(%)_lag5 -0.000814
f3_prInc_lag6     -0.000450
f10_p10_lag6       0.000458
f14_p90_lag6      -0.000693
SP_lg_ret(%)_lag6  0.001025

Test set evaluation:
MSE: 0.00110, RMSE: 0.03317, MAE: 0.02435, MAPE: 11.89264
